In [ ]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


In [ ]:
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (64, 64)) 
            labels.append(label)
    return images, labels

categories = ['dark', 'light', 'mid-dark', 'mid-light']
data = []
labels = []

for i, category in enumerate(categories):
    folder = f'./skintones/{category}'  
    images, category_labels = load_images_from_folder(folder, i)
    data.extend(images)
    labels.extend(category_labels)

data = np.array(data)
labels = np.array(labels)

In [ ]:
def extract_skin_color_features(images):
    features = []
    for img in images:
        img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        lower1 = np.array([0, 30, 60], dtype=np.uint8)
        upper1 = np.array([20, 150, 255], dtype=np.uint8)
        lower2 = np.array([160, 30, 60], dtype=np.uint8)
        upper2 = np.array([180, 150, 255], dtype=np.uint8)
        mask1 = cv2.inRange(img_hsv, lower1, upper1)
        mask2 = cv2.inRange(img_hsv, lower2, upper2)
        mask = cv2.bitwise_or(mask1, mask2)
        skin = cv2.bitwise_and(img, img, mask=mask)
        h_mean, h_std = np.mean(skin[:, :, 0]), np.std(skin[:, :, 0])
        s_mean, s_std = np.mean(skin[:, :, 1]), np.std(skin[:, :, 1])
        v_mean, v_std = np.mean(skin[:, :, 2]), np.std(skin[:, :, 2])
        features.append([h_mean, h_std, s_mean, s_std, v_mean, v_std])
    return np.array(features)


features = extract_skin_color_features(data)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy:.2f}')

In [ ]:
import joblib
joblib.dump(knn, 'knn_skin_tone_model.pkl')
print("Model saved as 'knn_skin_tone_model.pkl'.")

In [ ]:
def predict_skin_tone(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (64, 64))
    
    feature = extract_skin_color_features([img])
    prediction = knn.predict(feature)
    return prediction[0]

image_path = './testimg.jpg' 
predicted_skin_tone = predict_skin_tone(image_path)
print(f'Predicted Skin Tone: {categories[predicted_skin_tone]}')